## Project - LLM-Powered Clickbait Detector

Below are the instructions for the hands-on project explain in the video lecture. The goal is to build an LLM-powered clickbait detector:

Part 1: Design a prompt/chain that detects if an article is clickbait or not based on their headline. We have provided the article headlines along with their corresponding labels below. The first task is to convert those examples into a dataset. You will need to specify the instructions and the criteria for what a clickbait is in your prompt. 

Part 2: Use a moderation tool (e.g., OpenAI moderation APIs) to also classify whether the news articles contain harmful information or not. You also need to define what safe or unsafe is in your prompt. Feel free to use demonstrations or any of the approaches we discussed in the course.

Part 3: Experiment with GPT-3.5-Turbo for this task and log prompt + results using Comet's prompting tools. Use tags to label whether articles are safe/unsafe and clickbait/not clickbait. Use CoT, few-shot and zero-shot prompting techniques and compare performance.

Part 4: In the end, the goal should be to create a tagging system to label a set of articles as either safe/unsafe and clickbait/not clickbait. If the headline is unsafe or a clickbait, use GPT-3.5-Turbo or GPT-4 to rewrite the article as safe and that it doesn't contain clickbait. You can also try to log the results to Comet to properly debug and evaluate the results.


---

#### Headline examples and their categories

Feel free to add more headlines to the list

```
"This celebrity's diet secret will SHOCK you!" (Clickbait)
"You won't believe what happened next – video inside!" (Clickbait)
"Is the moon actually made of cheese? The truth revealed!" (Clickbait)
"Doctors are stunned by this one weird trick to lose weight!" (Clickbait)
"10 facts about your favorite show – Number 7 will blow your mind!" (Clickbait)
"NASA to Launch New Mars Rover Next Month." (Non-Clickbait)
"Federal Reserve Increases Interest Rates by 0.5%." (Non-Clickbait)
"Recent Studies Show Coffee May Have Health Benefits." (Non-Clickbait)
"A Comprehensive Review of the Latest Smartphone Models." (Non-Clickbait)
"UN Climate Summit to Begin in Paris Next Week." (Non-Clickbait)
"How to Make Homemade Explosives!" (Unsafe)
"The Secret Art of Digital Hacking – A Guide!" (Unsafe)
"10 Easy Ways to Commit Fraud and Get Rich Quick!" (Unsafe)
"Why Vaccines are the Biggest Scam in Medical History!" (Unsafe) (Note: This is a false and potentially harmful claim)
"Join the Underground Rebellion Against Government Control!" (Unsafe)
"10 celebrities who hate each other – Number 5 will make you gasp!" (Clickbait & Potentially Unsafe)
"Experts reveal the truth about eating raw meat." (Non-Clickbait but Potentially Unsafe if misinterpreted)
"The hidden dangers of everyday items in your home!" (Clickbait & Potentially Unsafe)
"Scientists discover groundbreaking method to cure all diseases!" (Clickbait) (Note: This is a misleading claim)
"The Untold Truth About the World's Secret Societies!" (Clickbait & Potentially Unsafe)
```

In [22]:
! pip install openai==0.28 comet-llm --quiet

In [3]:
import openai
import os
import IPython
import json
import pandas as pd
import numpy as np
import comet_llm


In [4]:
# API configuration
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai.api_key = os.environ.get("OPENAI_API_KEY")


The function below helps to generate the final results from the model after calling the OpenAI API:

In [5]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Load the Data

The code below loads both the few-shot demonstrations and the validation dataset used for testing the model.

In [6]:
import json

# Function to load JSON data from a file
def load_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Path to your JSON file
json_file_path = "filtered_original_clickbait_prompts.json"

# Load JSON data from file
clickbait_data = load_json_file(json_file_path)
print(len(clickbait_data))
print(clickbait_data)

13
[{'prompt': "You won't believe what happened next – video inside!\n\n###\n\n", 'tags': ['Clickbait']}, {'prompt': 'Is the moon actually made of cheese? The truth revealed!\n\n###\n\n', 'tags': ['Clickbait']}, {'prompt': 'Doctors are stunned by this one weird trick to lose weight!\n\n###\n\n', 'tags': ['Clickbait']}, {'prompt': '10 facts about your favorite show – Number 7 will blow your mind!\n\n###\n\n', 'tags': ['Clickbait']}, {'prompt': 'Federal Reserve Increases Interest Rates by 0.5%.\n\n###\n\n', 'tags': ['Non-Clickbait\n']}, {'prompt': 'Recent Studies Show Coffee May Have Health Benefits.\n\n###\n\n', 'tags': ['Non-Clickbait\n']}, {'prompt': 'A Comprehensive Review of the Latest Smartphone Models.\n\n###\n\n', 'tags': ['Non-Clickbait\n']}, {'prompt': 'UN Climate Summit to Begin in Paris Next Week.\n\n###\n\n', 'tags': ['Non-Clickbait\n']}, {'prompt': 'The Secret Art of Digital Hacking – A Guide!\n\n###\n\n', 'tags': ['Unsafe']}, {'prompt': '10 Easy Ways to Commit Fraud and Ge

In [7]:
# Path to your few_shot.json file
few_shot_file_path = "few_shot.json"

# Load JSON data from the few_shot.json file
few_shot_data = load_json_file(few_shot_file_path)

print(len(few_shot_data))
print(few_shot_data)


7
[{'prompt': "This celebrity's diet secret will SHOCK you!\n\n###\n\n", 'tags': ['Clickbait']}, {'prompt': 'NASA to Launch New Mars Rover Next Month.\n\n###\n\n', 'tags': ['Non-Clickbait']}, {'prompt': 'How to Make Homemade Explosives!\n\n###\n\n', 'tags': ['Unsafe']}, {'prompt': '10 celebrities who hate each other – Number 5 will make you gasp!\n\n###\n\n', 'tags': ['Clickbait', 'Potentially Unsafe']}, {'prompt': 'Scientists discover groundbreaking method to cure all diseases!\n\n###\n\n', 'tags': ['Clickbait', 'Misleading claim.']}, {'prompt': "The Untold Truth About the World's Secret Societies!\n\n###\n\n", 'tags': ['Clickbait', 'Potentially Unsafe']}, {'prompt': 'Experts reveal the truth about eating raw meat.\n\n###\n\n', 'tags': ['Non-Clickbait but Potentially Unsafe if misinterpreted\n']}]


The following is a helper function to obtain the final predictions from the model given a prompt template (e.g., zero-shot or few-shot) and the provided input data.

In [8]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        messages = messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]
        response = get_completion(messages)
        responses.append(response)

    return responses

### Few-Shot

First, we define a few-shot template which will leverage the few-shot demonstration data loaded previously.

In [9]:
# function to define the few-shot template
def get_clickbait_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n\n" + "\n".join([ "Headline: "+ ex["prompt"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# Define the few-shot prefix and suffix for clickbait detection
clickbait_few_shot_prefix = """
Your task is to determine if the following headlines are clickbait or not. Clickbait headlines\
typically use sensational language to grab attention or make exaggerated claims. They often promise surprising or\
shocking information to entice readers. Identify whether each headline falls into the category of clickbait or not. (e.g., ["Clickbait"]).
"""

clickbait_few_shot_suffix = """Headline: {input}\nTags:"""

# Function to sample few-shot data for clickbait detection
def sample_clickbait_data(data, n):
    return np.random.choice(few_shot_data, n, replace=False)

# Load 3 samples from the clickbait data
clickbait_few_shot_template = get_clickbait_few_shot_template(clickbait_few_shot_prefix, clickbait_few_shot_suffix, sample_clickbait_data(few_shot_data, 3))


In [10]:
clickbait_few_shot_template

'\nYour task is to determine if the following headlines are clickbait or not. Clickbait headlinestypically use sensational language to grab attention or make exaggerated claims. They often promise surprising orshocking information to entice readers. Identify whether each headline falls into the category of clickbait or not. (e.g., ["Clickbait"]).\n\n\nHeadline: This celebrity\'s diet secret will SHOCK you!\n\n###\n\n\nTags: [\'Clickbait\']\n\nHeadline: NASA to Launch New Mars Rover Next Month.\n\n###\n\n\nTags: [\'Non-Clickbait\']\n\nHeadline: 10 celebrities who hate each other – Number 5 will make you gasp!\n\n###\n\n\nTags: [\'Clickbait\', \'Potentially Unsafe\']\n\n\nHeadline: {input}\nTags:'

### Zero-Shot Template

The code below defines the zero-shot template. Note that we use the same instruction from the few-shot prompt template. But in this case, we don't use the demonstrations.

In [11]:
zero_shot_template = """
Your task is to determine if the following headlines are clickbait or not. Clickbait headlines\
typically use sensational language to grab attention or make exaggerated claims.\
They often promise surprising or shocking information to entice readers. Identify whether each headline falls into the\
category of clickbait or not. (e.g., ["Clickbait"])

Headline: {input}
Tags:
"""

### Get Predictions

We then generated all the predictions using the validation data as inputs:

In [12]:
headlines = [clickbait_data[i]["prompt"] for i in range(len(clickbait_data))]
headlines

["You won't believe what happened next – video inside!\n\n###\n\n",
 'Is the moon actually made of cheese? The truth revealed!\n\n###\n\n',
 'Doctors are stunned by this one weird trick to lose weight!\n\n###\n\n',
 '10 facts about your favorite show – Number 7 will blow your mind!\n\n###\n\n',
 'Federal Reserve Increases Interest Rates by 0.5%.\n\n###\n\n',
 'Recent Studies Show Coffee May Have Health Benefits.\n\n###\n\n',
 'A Comprehensive Review of the Latest Smartphone Models.\n\n###\n\n',
 'UN Climate Summit to Begin in Paris Next Week.\n\n###\n\n',
 'The Secret Art of Digital Hacking – A Guide!\n\n###\n\n',
 '10 Easy Ways to Commit Fraud and Get Rich Quick!\n\n###\n\n',
 'Why Vaccines are the Biggest Scam in Medical History!\n\n###\n\n',
 'Join the Underground Rebellion Against Government Control!\n\n###\n\n',
 'The hidden dangers of everyday items in your home!\n\n###\n\n']

In [13]:
few_shot_predictions = get_predictions(clickbait_few_shot_template, headlines)

Chain logged to https://www.comet.com/lorenamelos/llm-general


In [14]:
zero_shot_predictions = get_predictions(zero_shot_template, headlines)

In [15]:
expected_tags = [str(clickbait_data[i]["tags"]) for i in range(len(clickbait_data))]

In [16]:
print("Zero shot predictions")
print(zero_shot_predictions)
print("\n\nFew shot predictions")
print(few_shot_predictions)
print("\n\nExpected tags")
print(expected_tags)

Zero shot predictions
['["Clickbait"]', '["Clickbait"]', '["Clickbait"]', '["Clickbait"]', '["Not clickbait"]', '["Not clickbait"]', '["Not clickbait"]', '["Not Clickbait"]', '["Clickbait"]', 'Clickbait', '["Clickbait"]', '["Clickbait"]', '["Clickbait"]']


Few shot predictions
["['Clickbait']", "['Clickbait']", "['Clickbait']", "['Clickbait']", "['Non-Clickbait']", "['Non-Clickbait']", "['Non-Clickbait']", "['Non-Clickbait']", "['Non-Clickbait']", "['Clickbait', 'Potentially Unsafe']", "['Clickbait', 'Potentially Unsafe']", "['Non-Clickbait']", "['Clickbait']"]


Expected tags
["['Clickbait']", "['Clickbait']", "['Clickbait']", "['Clickbait']", "['Non-Clickbait\\n']", "['Non-Clickbait\\n']", "['Non-Clickbait\\n']", "['Non-Clickbait\\n']", "['Unsafe']", "['Unsafe']", "['Unsafe', 'This is a false and potentially harmful claim']", "['Unsafe']", "['Clickbait', 'Potentially Unsafe']"]


### Log Prompt Results

Finally, we log the prompt + results to Comet. We logging both the few-shot and zero-shot results, together with all the metadata and tags.

In [21]:
import comet_llm # type: ignore

# Comet Configuration
COMET_API_KEY = os.environ.get("COMET_API_KEY")
COMET_WORKSPACE = "COMET_WORKSPACE"

comet_llm.init(project="click-bait", api_key=COMET_API_KEY)


# log the predictions
for i in range(len(expected_tags)):
    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=clickbait_few_shot_template.format(input=headlines[i]),
        prompt_template=clickbait_few_shot_template,
        output=few_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "prompt": headlines[i]
        }
    )
     # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=headlines[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "prompt": headlines[i]
        }
    )

COMET INFO: Valid Comet API Key saved in /home/lorena/.comet.config (set COMET_CONFIG to change where it is saved).


## Chain-of-Though (CoT) Prompting

I used CoT to the final result. I believe with the step by step prompting it is possible to achieve the final result asked in this project.
First I tried the simple way above, but output proved to be very unstable. so I did as in the following example, in which I used the steps directly in the system.



### First attempt

In [42]:
# the system message contains the logic (step by step) for the system to follow

system_message = """

Instructions:

Step 1: Your task is to classify whether a prompt is a clickbait or not, considering if it is safe or not.
Consider that safety here refers to the absence of harmful or inappropriate content, including misinformation,
violence, hate speech, or any other form of content that could potentially cause harm or offense.\

Step 2: From step 1, now evaluate each headline based on its language, tone, and potential impact on readers
and identify whether each headline falls into the category of clickbait or not (e.g., ["Clickbait"]).
You can use few_shot_data as example to your reasoning.\

Step 3: If the headline contains any harmful elements or promotes unsafe behavior/clickbait, rewrite the headline as safe and that it doesn't contain clickbait\

Step 4: Prepare a response that contains the predicted tag and also the safe responde without clickbait.\


Return a response with the following reasoning steps and final output to the user:
Step 1: <Step 1 reasoning>
Step 2: <Step 2 reasoning>
Step 3: <Step 3 reasoning>
Step 4: <final response>
"""

messages = [
    {
        "role":"system",
        "content": system_message.format(clickbait_data=clickbait_data)
    },
    {
        "role":"system",
        "content": system_message.format(few_shot_data=few_shot_data)
    }
]

click_bait_response = get_completion(messages, temperature=0, max_tokens=500)
print(click_bait_response)

Sure, I can help with that. Please provide me with the headline you'd like me to evaluate.


### Second Attempt (sucessful)

In [46]:
# Define system message for instructions
system_message = """

Instructions:

Step 1: Your task is to classify whether a prompt is a clickbait or not, considering if it is safe or not.
Consider that safety here refers to the absence of harmful or inappropriate content, including misinformation,
violence, hate speech, or any other form of content that could potentially cause harm or offense.\

Step 2: From step 1, now evaluate each headline based on its language, tone, and potential impact on readers
and identify whether each headline falls into the category of clickbait or not (e.g., ["Clickbait"]).
You can use few_shot_data as an example for your reasoning.\

Step 3: If the headline contains any harmful elements or promotes unsafe behavior or clickbait,
rewrite the headline as safe and that it doesn't contain clickbait.\

Step 4: Prepare a response that contains the predicted tag and also the safe response without clickbait.\

Return a response with the following reasoning steps and final output to the user:
Step 1: <Step 1 reasoning>
Step 2: <Step 2 reasoning>
Step 3: <Step 3 reasoning>
Step 4: <final response>
"""

# Define messages for each step
step1_message = {
    "role": "system",
    "content": "Step 1: Classify whether the prompt is a clickbait or not, considering safety criteria."
}

step2_message = {
    "role": "system",
    "content": "Step 2: Evaluate each headline for language, tone, and potential impact to determine clickbait status."
}

step3_message = {
    "role": "system",
    "content": "Step 3: Rewrite headlines containing harmful elements or promoting unsafe behavior as safe."
}

step4_message = {
    "role": "system",
    "content": "Step 4: Prepare a response with the original prompt, the predicted tag and the rewritten safe response."
}

# Combine messages
messages = [step1_message, step2_message, step3_message, step4_message]

# Get completion based on messages
click_bait_response = get_completion(messages, temperature=0, max_tokens=500)

# Print the response
print(click_bait_response)


**Original prompt:** "10 Dangerous Foods You Should Never Eat"

**Predicted tag:** Clickbait

**Rewritten safe response:** "10 Foods to Avoid for Better Health"
